+ 分组作业   1300017640王瑞康 1300017639吴振宇 1300017674何雨凡
 +  行业 ：房地产

In [21]:
##________________________________________set up the database  ——————————————————————————————————————————————————
import sqlite3
con= sqlite3.connect('teamwork102.db')  

In [22]:
##____________________________________ set the environment ____________________________________________________________
import os
import sys
import numpy as np
from numpy.random import randn
from pandas import Series, DataFrame
import pandas.io.sql as sql
import pandas as pd
import matplotlib.pyplot as plt
from os.path import getsize
from urllib.request import urlretrieve
from lxml.html import parse
from urllib.request import urlopen
import urllib 
from __future__ import division
from pandas.io.parsers import TextParser

In [23]:
%pwd

'/Users/wrk/Documents/notebook0'

In [24]:
##_________________________________________get sina high_______________________________________________________________
def get_sina_high(stockid,start,end):
    #stockid with pre sh or sz
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="B")

    for cq in daterange:
        url='http://market.finance.sina.com.cn/downxls.php?date='+str(cq.date())+'&symbol='+stockid
        urllib.urlretrieve(url, "temp.txt")    
        # if no data, temp.txt just contains few words        
        if getsize("temp.txt")<100:
            continue
        HighF=pd.read_table("temp.txt",na_values="--",encoding="gbk",index_col=0,parse_dates=True)
        HighF=HighF.ix[range(len(HighF)-1,-1,-1)]
        delta=cq.date()-HighF.index.date[0]
        HighF.index=HighF.index+delta
        data=pd.concat([data,HighF])
    return data


In [25]:
##________________________get_sina_stock(with stock id)______________________________________________________________
def get_sina_stock(stockid,start="1990-12-19",end=pd.datetime.now(),isstock=True,fuquan=True):    
    def _unpack(row, kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符    
    def parse_options_data(table,isstock,fuquan):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[2:]]
        if isstock & fuquan:
            colnames = ['date','open','high','close','low','vol','amount','ratio'] # 回避中文处理
        else:
            colnames = ['date','open','high','close','low','vol','amount']
        data= pd.DataFrame(data,columns=colnames)
        data.index=pd.to_datetime(data["date"])
        data=data.drop("date",axis=1)
        data=data.ix[range(len(data)-1,-1,-1)]
        return  data
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="Q")
    daterange=daterange.insert(len(daterange),daterange[-1]+1)
    stockid=str(stockid).zfill(6)
    for cq in daterange: 
        if isstock:
            if fuquan:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_FuQuanMarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
            else:
                url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)            
        else:            
            url='http://vip.stock.finance.sina.com.cn/corp/go.php/vMS_MarketHistory/stockid/'+str(stockid)+'/type/S.phtml?year='+str(cq.year)+'&jidu='+str(cq.month/3)
        try:
            parsed = parse(url)
        except:
            print("download failed for year=" +str(cq.year)+',  jidu='+str(cq.month/3))
            continue
        doc = parsed.getroot()
        tables=doc.findall('.//table')     
        da= tables[-1] # last table 19 for fuquan data and 4 for index
        datatem = parse_options_data(da,isstock,fuquan)
        data=pd.concat([data,datatem])
    return pd.DataFrame(data[start:end],dtype=float)    


In [26]:
##_____________________________________  get_sina_high ______________________________________________________________

def get_sina_high(stockid,start,end):
    #stockid with pre sh or sz
    data=pd.DataFrame()
    daterange=pd.date_range(start,end,freq="B")
    for cq in daterange:
        url='http://market.finance.sina.com.cn/downxls.php?date='+str(cq.date())+'&symbol='+stockid
        urllib.urlretrieve(url, "temp.txt")    
        # if no data, temp.txt just contains few words        
        if getsize("temp.txt")<100:
            continue
        HighF=pd.read_table("temp.txt",na_values="--",encoding="gbk",index_col=0,parse_dates=True)
        HighF=HighF.ix[range(len(HighF)-1,-1,-1)]
        delta=cq.date()-HighF.index.date[0]
        HighF.index=HighF.index+delta
        data=pd.concat([data,HighF])
    return data



In [27]:
##______________________________________ get financial related data  _________________________________________________

def get_sina_fin(stockid,year):
    def _unpack(row,kind='td'):
        elts = row.findall('.//%s' % kind)
        return [val.text_content().strip() for val in elts] # .strip()去掉\r\t\n之类的字符 
    def parse_options_data(table):
        rows = table.findall('.//tr')
        data = [_unpack(r) for r in rows[1:]]    
        data= pd.DataFrame(data)
        
        datacol=pd.to_datetime(data.ix[0][1:])
        dataindex=data[0][1:]        
        data=data.drop(0,axis=1)
        data=data.ix[1:]
        data.index=dataindex
        data.columns=datacol
        return  data

    url="http://money.finance.sina.com.cn/corp/go.php/vFD_ProfitStatement/stockid/"+str(stockid)+"/ctrl/"+str(year)+"/displaytype/4.phtml"
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
    tables=doc.findall('.//table')
    da= tables[-2]      
    datatem = parse_options_data(da)
     
    return datatem


In [28]:
##——————————————————————————————————————————————————————import stockid list___________________________________________

stockid_list = pd.read_csv('data/realstate.csv')
IDlist = []
for x in  stockid_list['stockid']:
    y = x[:6]
    IDlist.append(y)

print(IDlist)

['600376', '601155', '000926', '600077', '000062', '600684', '002344', '000838', '000615', '000809', '600510', '600162', '600622', '600393', '600466', '600503', '600515', '600606', '600565', '600604', '600621', '600533', '600555', '600223', '600215', '600234', '600225', '600177', '600173', '600208', '600185', '600340', '600325', '600383', '000006', '600246', '600239', '600322', '600266', '600638', '600773', '600753', '600791', '600790', '600745', '600743', '600748', '600747', '000005', '600895', '000002', '601588', '600823', '600807', '600890', '600848', '600663', '600658', '600675', '600665', '600641', '600639', '600657', '600649', '600732', '600724', '600736', '600733', '600692', '600683', '600716', '600708', '000608', '000573', '000616', '000609', '000540', '000537', '000567', '000558', '000671', '000668', '000718', '000691', '000631', '000620', '000667', '000656', '000043', '000042', '000056', '000046', '000031', '000029', '000040', '000036', '000514', '000506', '000534', '000517',

In [29]:
##__________________________________________import daily data to the database__________________________________________

for stockid in IDlist:
    x=get_sina_stock(stockid,'2011-10-20','2016-10-20')
    ##reload(sys)
    ##sys.setdefaultencoding( "utf-8" )
    sql.to_sql(x,'daily'+str(stockid),con)

In [30]:
##_________________________________________import daily high frequency data to the database____________________________

for stockid in IDlist:
    try:
        x=get_sina_high(stockid_2,'2011-10-20','2016-10-20')
        ##reload(sys)
        ##sys.setdefaultencoding( "utf-8" )
        sql.to_sql(x,'high'+str(stockid),con)
    except:
        print('Not available' + str(stockid))
        continue

Not available600376
Not available601155
Not available000926
Not available600077
Not available000062
Not available600684
Not available002344
Not available000838
Not available000615
Not available000809
Not available600510
Not available600162
Not available600622
Not available600393
Not available600466
Not available600503
Not available600515
Not available600606
Not available600565
Not available600604
Not available600621
Not available600533
Not available600555
Not available600223
Not available600215
Not available600234
Not available600225
Not available600177
Not available600173
Not available600208
Not available600185
Not available600340
Not available600325
Not available600383
Not available000006
Not available600246
Not available600239
Not available600322
Not available600266
Not available600638
Not available600773
Not available600753
Not available600791
Not available600790
Not available600745
Not available600743
Not available600748
Not available600747
Not available000005
Not available600895


In [31]:
a = get_sina_high('600325','2016-10-18','2016-10-21')
print(a.dtype)

AttributeError: module 'urllib' has no attribute 'urlretrieve'

In [ ]:
##________________________________unpack the data from website_____________________________________________________
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content().strip() for val in elts]# .strip()去掉\r\t\n之类的字符   

##_____________________________________define get_data_____________________________________________________________
def get_data(url,stockname,dataamounts = 12,timelenth=23):
    parsed = parse(urlopen(url))
    doc = parsed.getroot()
    tables=doc.findall('.//table')
    table = tables[19]
    row = table.findall('.//tr')
    data = [_unpack(r) for r in row[1:]]
    
    flag = 0
    tot = (dataamounts+1)*timelenth+1
    for i in range(1,min(tot,len(row)),dataamounts+1):
        header=_unpack(row[i],kind="td")
        header = [stockname,header[1]]
        x = TextParser(data[i:i+dataamounts-1], names=header).get_chunk()
        x = x.set_index([header[0]])
        if flag ==0:
            y = x
            flag = 1
        else:
            y = y.join(x,how='outer')
    y = y.T
    return y

In [ ]:

##_________________________________________eliminate the error when dealing with 0_____________________________________
def str2stockid(x):
    return (6-len(str(x)))*'0'+str(x)
##here is a test:
##print(str2stockid('000003'))

##________________________________________add financial data to the dtabase___________________________________________ 
for x in IDlist1:
    stockid = str2stockid(x)
    ##url of financial information:
    url = 'http://vip.stock.finance.sina.com.cn/corp/go.php/vFD_FinanceSummary/stockid/'+stockid+'.phtml'
    stockname = stockid + 'accounting_files'
    y = get_data(url,stockname)
    print(y)
    try:
        y.to_sql(stockname,con)
    except:
        print('Stock_id Overlap')